In [1]:
import pandas as pd
import dtale
# helps convert zip to lat and long
from pyzipcode import ZipCodeDatabase

# import data

In [2]:

#absent
absent_df=pd.read_excel('../data/original_data/reportcarddata2019.xlsx','chronic_absenteeism'
                    )
# import finance data only need 141 rows the rest are missing
finance_school = pd.read_excel('../data/original_data/reportcarddata2019.xlsx','finance'
                        )

#district info

district_df = pd.read_excel('../data/original_data/reportcarddata2019.xlsx','district_info'
                              
                              )
# info on schools
school_df = pd.read_excel('../data/original_data/reportcarddata2019.xlsx','school_info'
                              
                             )
# grade data
grade_df = pd.read_excel('../data/original_data/reportcarddata2019.xlsx','overall_grades'
                              )
# data on teacher
teacher_df = pd.read_excel('../data/original_data/reportcarddata2019.xlsx','teacher_retention'
                              
                           )

# data on enrollment
enrollment_df = pd.read_excel('../data/original_data/reportcarddata2019.xlsx','enrollment'
                              
                              )
# data on staff
staffing_df = pd.read_excel('../data/original_data/reportcarddata2019.xlsx','staffing'
                              
                              )

# clean

# School cleaning

In [3]:
zipcode_database = ZipCodeDatabase()

In [4]:
def get_long(zip):
    try:
        return zipcode_database[zip].longitude
    except:
        pass
def get_lat(zip):
    try:
        return zipcode_database[zip].latitude
    except:
        pass
school_df['longitude'] = school_df['zipcode'].apply(get_long)
school_df['latitude'] = school_df['zipcode'].apply(get_lat)

In [5]:
school_df =school_df[['school_name','zipcode','longitude','latitude']]

# absent cleaning

In [6]:
#absent_df

In [7]:
absent_df=absent_df[absent_df.subgroup =='all_students']

In [8]:
absent_df=absent_df[absent_df.school_name !='All Schools']

In [9]:
absent_df=absent_df[['district_name','school_name','percent_ca']]

In [10]:
absent_df=absent_df[absent_df['percent_ca'] != '*']

In [11]:
absent_df=absent_df[absent_df['percent_ca'] != '**']

In [12]:
absent_df['percent_ca']=absent_df['percent_ca'].astype(float)

In [13]:
absent_df.dtypes

district_name     object
school_name       object
percent_ca       float64
dtype: object

# grade cleaning

In [14]:
grade_df=grade_df[['school_name','pool','score_achievement',]]

In [15]:
grade_df= grade_df.dropna()

In [16]:
#grade_df.dtypes

# teacher cleaning

In [17]:
teacher_df= teacher_df[['school_name','percent_retained']]

In [18]:
teacher_df=teacher_df[teacher_df.school_name !='All Schools']

In [19]:
teacher_df=teacher_df.dropna()

In [20]:
teacher_df=teacher_df[teacher_df['percent_retained'] != '**']

In [21]:
teacher_df=teacher_df[teacher_df['percent_retained'] != '*']

In [22]:
teacher_df['percent_retained']=teacher_df['percent_retained'].astype(float)

In [23]:
#d = dtale.show(teacher_df)
#d.open_browser()

# staff cleaning

In [24]:
staffing_df=staffing_df[['school_name','n_staff','role']]

In [25]:
staffing_df=staffing_df[staffing_df.school_name !='All Schools']

In [26]:
staffing_df=staffing_df[staffing_df.role =='teacher']

In [27]:
staffing_df= staffing_df.dropna()

In [28]:
staffing_df=staffing_df.rename(columns={'n_staff':'number_of_teachers'})

In [29]:
staffing_df=staffing_df.drop(['role'],axis=1)

In [30]:
#staffing_df

# enrollment

In [34]:
enrollment_df=enrollment_df[enrollment_df.subgroup =='all_students']

In [38]:
enrollment_df=enrollment_df[enrollment_df.school_name !='All Schools']

In [40]:
enrollment_df=enrollment_df[['district_name','school_name','value']]

In [42]:
enrollment_df=enrollment_df.rename(columns={'value':'students_enrolled'})

In [44]:
#enrollment_df

# join data

In [ ]:
'''staffing_df
absent_df
teacher_df
school_df
grade_df
enrollment_df'''

In [46]:
# merge school and and enrollment
#fd_df=district_df.merge(finance_school,how='inner', on="district_name")
merge_se=school_df.merge(enrollment_df,how='inner',on='school_name')
merge_se.shape

(1956, 6)

In [48]:
# merge_se with staffing
merge_ses= merge_se.merge(staffing_df,how='inner',on='school_name')
merge_ses.shape

(2625, 7)

In [50]:
merge_ses=merge_ses.drop_duplicates(subset=['school_name'])

In [53]:
# merge_ses with absent
merge_sesa=merge_ses.merge(absent_df,how='inner',on='school_name')
merge_sesa.shape

(1724, 9)

In [55]:
# merge_sesa with teacher
merge_sesat=merge_sesa.merge(teacher_df,how='inner',on='school_name')
merge_sesat.shape

(1893, 10)

In [57]:
# merge_sesa with grade
merge_sesatg=merge_sesat.merge(grade_df,how='inner',on='school_name')
merge_sesatg.shape

(2510, 12)

In [59]:
merge_sesatg=merge_sesatg.drop_duplicates(subset=['school_name'])

In [60]:
'''d = dtale.show(merge_sesatg)
d.open_browser()'''

2021-06-07 15:50:23,787 - INFO     - Patching spark automatically. You can disable it by setting SPARK_KOALAS_AUTOPATCH=false in your environment


# finance cleaning

In [64]:
finance_school=finance_school[['district_name','district_ppe','local_funding_percent','state_funding_percent','federal_funding_percent']]

In [ ]:
finance_school=finance_school.dropna()

# district clean

In [71]:
district_df=district_df[['district_name','zipcode','countyname','street_address_1','city']]

In [74]:
district_df=district_df[district_df.countyname !='State']

In [76]:
district_df['longitude'] = district_df['zipcode'].apply(get_long)
district_df['latitude'] = district_df['zipcode'].apply(get_lat)

<ipython-input-76-66721e53a61f>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-76-66721e53a61f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [83]:
# merge district and finance
merge_fd=district_df.merge(finance_school,how='inner',on='district_name')
merge_fd.shape

(141, 11)

In [85]:
merge_fd=merge_fd.drop_duplicates(subset=['district_name'])

In [88]:
# drop data that is not needed to merge with finance and district
merge_sesatg_droped=merge_sesatg.drop(columns={'zipcode','longitude','latitude','school_name','students_enrolled','number_of_teachers','district_name_y','pool'})

In [91]:
merge_sesatg_droped=merge_sesatg_droped.rename(columns={'district_name_x':'district_name'})

In [93]:
# groupby
merge_sesatg_droped=merge_sesatg_droped.groupby(['district_name']).mean()

Executing shutdown...


2021-06-07 16:45:47,636 - INFO     - Executing shutdown...


In [94]:
merge_sesatg_droped.shape

(142, 3)

In [ ]:
# merge by district_name

In [96]:
merge_fds= merge_sesatg_droped.merge(merge_fd,how='inner',on='district_name')

In [97]:
d = dtale.show(merge_fds)
d.open_browser()